# Interacting with Recommenders and Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Using Static and Dynamic Filters](#filters)
1. [Real-time Events](#real-time)
1. [Batch Recommendations](#batch)
1. [Wrap Up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 


In this Notebook we will interact with Recommenders and Campaigns and get recommendatiosn. We will interact with filters and send live data to Amazon Personalize to see the effect on recommendations.

![Workflow](images/image3.png)

To run this notebook, you need to have run the previous notebooks, `01_Data_Layer.ipynb`, and `02_Training_Layer.ipynb`, where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions and campaigns. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

As you work with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


First, let's create a supporting function to help make sense of the results returned by a Personalize recommender or campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the Movielens dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return a movie by just querying the ID. Movie #589 should be Terminator 2: Judgment Day.

In [5]:
movieIdExample = 589
title = items_df.loc[movieIdExample]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movieId, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movieId)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [7]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movieId="1197"))
# A bad type of value
print(get_movie_by_id(movieId="987.9393939"))
# Really bad values
print(get_movie_by_id(movieId="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

## Interact with recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "More like X" Recommender

'More like X' requires an item and a user as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from the "More like X" Recommender and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

We will be using the `recommenderArn`, the `itemId`, the `userId` as well as the number or results we want, `numResults`.

In [8]:
# First pick a user
testUserId = "1"

In [9]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_more_like_x_arn,
    itemId = str(589),
    userId = testUserId,
    numResults = 20
)

In [10]:
itemList = get_recommendations_response['itemList']
for item in itemList:
    print(get_movie_by_id(movieId=item['itemId']))

Independence Day (a.k.a. ID4) (1996)
Lion King, The (1994)
Speed (1994)
Babe (1995)
Hunt for Red October, The (1990)
Fugitive, The (1993)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Firm, The (1993)
Jurassic Park (1993)
Indiana Jones and the Last Crusade (1989)
Quiz Show (1994)
Mrs. Doubtfire (1993)
Star Wars: Episode VI - Return of the Jedi (1983)
Die Hard (1988)
Ghost (1990)
True Lies (1994)
Maverick (1994)
Fifth Element, The (1997)
Natural Born Killers (1994)
Dave (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar movies render in a nice dataframe. Again, let's create a helper function to achieve this.

In [11]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_id, user_id):
    # Get the movie name
    movie_name = get_movie_by_id(movie_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommender_more_like_x_arn,
        itemId = str(movie_id),
        userId = user_id
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our "More like X" Recommender. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [12]:
samples = items_df.sample(5)
samples

,title
movieId,
5993,Max (2002)
3265,Hard-Boiled (Lat sau san taam) (1992)
27904,"Scanner Darkly, A (2006)"
45382,Down in the Valley (2005)
2096,Sleeping Beauty (1959)


In [13]:
more_like_x_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    more_like_x_recommendations_df = get_new_recommendations_df(more_like_x_recommendations_df, movie, testUserId)

more_like_x_recommendations_df

,Max (2002),Hard-Boiled (Lat sau san taam) (1992),"Scanner Darkly, A (2006)",Down in the Valley (2005),Sleeping Beauty (1959)
0,Star Wars: Episode II - Attack of the Clones (...,Menace II Society (1993),Idiocracy (2006),Harry Potter and the Prisoner of Azkaban (2004),"Little Mermaid, The (1989)"
1,Interstate 60 (2002),All About Eve (1950),Sunshine (2007),Harry Potter and the Goblet of Fire (2005),"Lion King, The (1994)"
2,"Matrix Reloaded, The (2003)",Apollo 13 (1995),Waking Life (2001),"Matrix Revolutions, The (2003)",Cinderella (1950)
3,Spider-Man 2 (2004),"Fugitive, The (1993)",Natural Born Killers (1994),Charlie and the Chocolate Factory (2005),"Shawshank Redemption, The (1994)"
4,Clockstoppers (2002),Jurassic Park (1993),"Dark Knight, The (2008)",Spider-Man 2 (2004),Vertigo (1958)
5,Jurassic Park III (2001),Forrest Gump (1994),Inception (2010),Harry Potter and the Order of the Phoenix (2007),Sixteen Candles (1984)
6,"Matrix Revolutions, The (2003)","Net, The (1995)","Matrix Reloaded, The (2003)","Matrix Reloaded, The (2003)",Victor/Victoria (1982)
7,Planet of the Apes (2001),Independence Day (a.k.a. ID4) (1996),Avatar (2009),Batman Begins (2005),"Miracle of Morgan's Creek, The (1944)"
8,Spider-Man (2002),Braveheart (1995),"Matrix Revolutions, The (2003)",Watchmen (2009),"Dark Crystal, The (1982)"
9,"Adventures of Pluto Nash, The (2002)",Demolition Man (1993),Watchmen (2009),Star Wars: Episode II - Attack of the Clones (...,"Little Drummer Boy, The (1968)"


You may notice that some of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). 

### "Top picks for you" Recommender

"Top picks for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since "Top picks for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [14]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[242, 557, 554]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Top picks for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a movie.

The cells below will handle getting recommendations from the "Top picks for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [15]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommender_top_picks_arn,
        userId = str(user_id),
        numResults = 20
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [16]:
recommendations_df_users = pd.DataFrame()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,242,557,554
0,"Firm, The (1993)",Iron Man (2008),Groundhog Day (1993)
1,Nell (1994),Up (2009),"Nutty Professor, The (1996)"
2,"Specialist, The (1994)",Fight Club (1999),Brazil (1985)
3,What's Eating Gilbert Grape (1993),"Dark Knight, The (2008)",Beetlejuice (1988)
4,Ghost (1990),"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...","Fugitive, The (1993)"
5,Outbreak (1995),Inglourious Basterds (2009),Labyrinth (1986)
6,GoldenEye (1995),X-Men (2000),"Nightmare Before Christmas, The (1993)"
7,"Net, The (1995)",Ocean's Eleven (2001),Like Water for Chocolate (Como agua para choco...
8,"Client, The (1994)",Gladiator (2000),Powder (1995)
9,Speed (1994),Donnie Darko (2001),Star Trek: First Contact (1996)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)

Now that the reranking campaign is deployed and active, we can start to get recommendations via an API call. 

### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period, etc...). This may not be information that you have in your metadata, so an item metadata filter will not work, however you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [17]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [18]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Adam and Eve (National Lampoon's Adam & Eve) (...
1,Desperate Living (1977)
2,"Bank Dick, The (1940)"
3,"King's Speech, The (2010)"
4,Margin Call (2011)
5,"Aviator, The (2004)"
6,"Big Doll House, The (1971)"
7,Partly Cloudy (2009)
8,November (2004)
9,Chariots of Fire (1981)


Then make the personalized ranking API call.

In [19]:
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = str(rerank_user),
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [20]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Adam and Eve (National Lampoon's Adam & Eve) (...,Swept Away (Travolti da un insolito destino ne...
1,Desperate Living (1977),Chariots of Fire (1981)
2,"Bank Dick, The (1940)",Citizen Kane (1941)
3,"King's Speech, The (2010)","Handmaid's Tale, The (1990)"
4,Margin Call (2011),Desperate Living (1977)
5,"Aviator, The (2004)",Eddie Murphy Raw (1987)
6,"Big Doll House, The (1971)",Fall (1997)
7,Partly Cloudy (2009),Deathtrap (1982)
8,November (2004),"Man Without a Past, The (Mies vailla menneisyy..."
9,Chariots of Fire (1981),Glitter (2001)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Using Static and Dynamic Filters <a class="anchor" id="filters"></a>
[Back to top](#top)

Lets interact with the static filters we created in the previous notebook, and utilize dynamic filters in realtime.

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata (that arent range based) - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.

Now lets apply item filters to see recommendations for one of these users within each decade of our static filters.


In [21]:
def get_new_recommendations_df_by_static_filter(recommendations_df, user_id, filter_arn):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [22]:
def get_new_recommendations_df_by_dynamic_filter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommender_top_picks_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = genre_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given decade. Within a VOD application you could create Shelves (also known as rails or carousels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [23]:
recommendations_df_decade_shelves = pd.DataFrame()
for filter_arn in meta_filter_decade_arns:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_static_filter(recommendations_df_decade_shelves, user, filter_arn)

recommendations_df_decade_shelves

,1950s,1960s,1970s,1980s,1990s,2000s,2010s
0,"20,000 Leagues Under the Sea (1954)",8 1/2 (8Â½) (1963),Willy Wonka & the Chocolate Factory (1971),Brazil (1985),Groundhog Day (1993),Dune (2000),Holy Motors (2012)
1,This Island Earth (1955),2001: A Space Odyssey (1968),"Christmas Carol, A (1977)",Beetlejuice (1988),"Nutty Professor, The (1996)",Avalon (2001),Eva (2011)
2,Brigadoon (1954),Mothra (Mosura) (1961),Carrie (1976),Labyrinth (1986),"Fugitive, The (1993)",Glitter (2001),"Zero Theorem, The (2013)"
3,"Angry Red Planet, The (1959)",Planet of the Apes (1968),Taxi Driver (1976),Field of Dreams (1989),"Nightmare Before Christmas, The (1993)",Big Fish (2003),"Brass Teapot, The (2012)"
4,Harvey (1950),Night of the Living Dead (1968),Scrooge (1970),Big (1988),Like Water for Chocolate (Como agua para choco...,Eternal Sunshine of the Spotless Mind (2004),Marvel One-Shot: Item 47 (2012)
5,"Incredible Shrinking Man, The (1957)",Yellow Submarine (1968),All That Jazz (1979),"Princess Bride, The (1987)",Powder (1995),Supernova (2000),Byzantium (2012)
6,"Atomic Submarine, The (1959)","Birds, The (1963)",Young Frankenstein (1974),Time Bandits (1981),Star Trek: First Contact (1996),Mission to Mars (2000),Escape From Tomorrow (2013)
7,Gigi (1958),Fellini Satyricon (1969),"Rocky Horror Picture Show, The (1975)","Fish Called Wanda, A (1988)","Santa Clause, The (1994)","Fast Runner, The (Atanarjuat) (2001)",Ruby Red (2013)
8,"Day the Earth Stood Still, The (1951)",Sympathy for the Devil (1968),Alien (1979),"Fly, The (1986)","Mask, The (1994)",Simone (S1m0ne) (2002),"Bay, The (2012)"
9,"Christmas Carol, A (Scrooge) (1951)",Dr. Strangelove or: How I Learned to Stop Worr...,Sleeper (1973),RoboCop (1987),Pleasantville (1998),Tuck Everlasting (2002),"Hobbit: An Unexpected Journey, The (2012)"


In [24]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRES,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,1640995200
2,Adventure|Children|Fantasy,1995,1640995200
3,Comedy|Romance,1995,1640995200
4,Comedy|Drama|Romance,1995,1640995200
5,Comedy,1995,1640995200
6,Action|Crime|Thriller,1995,1640995200
7,Comedy|Romance,1995,1640995200
8,Adventure|Children,1995,1640995200
9,Action,1995,1640995200


Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [25]:
unique_genre_field_values = items_meta_df['GENRES'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [26]:
genres_to_filter

['Sci-Fi',
 'Horror',
 'Mystery',
 'Children',
 'Animation',
 'Drama',
 'IMAX',
 'Film-Noir',
 'Western',
 'Romance',
 'Thriller',
 'Adventure',
 'Documentary',
 'Comedy',
 'Musical',
 'Fantasy',
 'Action',
 'Crime',
 'War']

In [27]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_dynamic_filter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

,Sci-Fi,Horror,Mystery,Children,Animation,Drama,IMAX,Film-Noir,Western,Romance,Thriller,Adventure,Documentary,Comedy,Musical,Fantasy,Action,Crime,War
0,"Nutty Professor, The (1996)",Dracula (Bram Stoker's Dracula) (1992),"City of Lost Children, The (CitÃ© des enfants ...","Nightmare Before Christmas, The (1993)","Nightmare Before Christmas, The (1993)",Like Water for Chocolate (Como agua para choco...,Titanica (1992),L.A. Confidential (1997),Back to the Future Part III (1990),Groundhog Day (1993),"Fugitive, The (1993)",Labyrinth (1986),Hoop Dreams (1994),"Nutty Professor, The (1996)",Labyrinth (1986),Groundhog Day (1993),Star Trek: First Contact (1996),True Romance (1993),Forrest Gump (1994)
1,Brazil (1985),"Fly, The (1986)",Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Field of Dreams (1989),Toy Story (1995),Powder (1995),Africa: The Serengeti (1994),Dark City (1998),Maverick (1994),"Nutty Professor, The (1996)",True Romance (1993),Star Trek: First Contact (1996),Dancemaker (1998),Groundhog Day (1993),"Nightmare Before Christmas, The (1993)","Nutty Professor, The (1996)","Mask, The (1994)","Mask, The (1994)",Captain America (1990)
2,Powder (1995),Evil Dead II (Dead by Dawn) (1987),Strange Days (1995),Willy Wonka & the Chocolate Factory (1971),James and the Giant Peach (1996),"Santa Clause, The (1994)",Beauty and the Beast (1991),Lost Highway (1997),Unforgiven (1992),Like Water for Chocolate (Como agua para choco...,Star Trek: First Contact (1996),"City of Lost Children, The (CitÃ© des enfants ...",Hearts of Darkness: A Filmmakers Apocalypse (1...,Beetlejuice (1988),Willy Wonka & the Chocolate Factory (1971),Brazil (1985),"Princess Bride, The (1987)",Fargo (1996),"English Patient, The (1996)"
3,Star Trek: First Contact (1996),Carrie (1976),Naked Lunch (1991),Toy Story (1995),Memories (MemorÃ®zu) (1995),"Firm, The (1993)",Everest (1998),Devil in a Blue Dress (1995),Tombstone (1993),Big (1988),"Firm, The (1993)",Time Bandits (1981),Madonna: Truth or Dare (1991),"Santa Clause, The (1994)",James and the Giant Peach (1996),Beetlejuice (1988),Armageddon (1998),"Fish Called Wanda, A (1988)",Platoon (1986)
4,"City of Lost Children, The (CitÃ© des enfants ...",Leprechaun 4: In Space (1997),"Client, The (1994)",Addams Family Values (1993),Fantasia (1940),Field of Dreams (1989),Alaska: Spirit of the Wild (1997),Suture (1993),Wild Wild West (1999),Dracula (Bram Stoker's Dracula) (1992),Fargo (1996),"Princess Bride, The (1987)",Visions of Light: The Art of Cinematography (1...,"Mask, The (1994)",Madonna: Truth or Dare (1991),Labyrinth (1986),Captain America (1990),Heat (1995),Rob Roy (1995)
5,Contact (1997),Species (1995),"X-Files: Fight the Future, The (1998)","Honey, I Shrunk the Kids (1989)",Toy Story 2 (1999),Pleasantville (1998),More (1998),One False Move (1992),"Last of the Mohicans, The (1992)","Fisher King, The (1991)","Crying Game, The (1992)",Total Recall (1990),Crumb (1994),Pleasantville (1998),Farinelli: il castrato (1994),"Nightmare Before Christmas, The (1993)",Total Recall (1990),"Crow, The (1994)",Braveheart (1995)
6,Deep Impact (1998),Puppet Master III: Toulon's Revenge (1991),Primal Fear (1996),James and the Giant Peach (1996),Wicked City (YÃ´jÃ» toshi) (1987),Being John Malkovich (1999),Apollo 13 (1995),Miller's Crossing (1990),Legends of the Fall (1994),Forrest Gump (1994),Dracula (Bram Stoker's Dracula) (1992),"Dark Crystal, The (1982)",For All Mankind (1989),Being John Malkovich (1999),Fantasia (1940),Like Water for Chocolate (Como agua para choco...,Heat (1995),RoboCop (1987),Richard III (1995)
7,Gattaca (1997),Leprechaun 2 (1994),JFK (1991),FairyTale: A True Story (1997),Ghost in the Shell (KÃ´kaku kidÃ´tai) (1995),"City of Lost Children, The (CitÃ© des enfants ...",T-Rex: Back to the Cretaceous (1998),Bitter Moon (1992),The Beast of Hollow Mountain (1956),Ghost (1990),Deep Impact (1998),Toy Story (1995),Return with Honor (1998),Big (1988),Scrooge (1970),"Santa Clause, The (1994)","Crow, The (1994)",Sneakers (1992),Crim

## Real-time Events<a class="anchor" id="real-time"></a>
[Back to top](#top)

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

Start by creating an event tracker that is attached to the dataset group. This event tracker will add information to the dataset and will influence the recommendations.

In [28]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
trackingId = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:131578276461:event-tracker/d8bc4001
11bcce35-6bd3-496b-8c49-b059424b2df0


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [29]:
sessionDict = {}

def send_movie_click(userId, itemId, eventType):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        sessionId = sessionDict[str(userId)]
    except:
        sessionDict[str(userId)] = str(uuid.uuid1())
        sessionId = sessionDict[str(userId)]
        
    # Configure Properties:
    event = {
    "itemId": str(itemId),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = trackingId,
    userId= str(userId),
    sessionId = sessionId,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(eventType),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, userId, itemId, eventType):
    # Get the artist name (header of column)
    movieName = get_movie_by_id(itemId)
    
    # Interact with different movies
    print('sending event ' + eventType + ' for ' + get_movie_by_id(itemId))
    send_movie_click(userId=userId, itemId=itemId,eventType=eventType)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommender_top_picks_arn,
        userId = str(userId),
    )
    # Build a new dataframe of recommendations
    itemList = get_recommendations_response['itemList']
    recommendation_list = []
    for item in itemList:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [movieName])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [30]:
# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommender_top_picks_arn,
        userId = str(rerank_user),
    )

# Build a new dataframe for the recommendations
itemList = get_recommendations_response['itemList']
recommendationList = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendationList.append(artist)
user_recommendations_df = pd.DataFrame(recommendationList, columns = [rerank_user])
user_recommendations_df

,554
0,Swept Away (Travolti da un insolito destino ne...
1,Chariots of Fire (1981)
2,Citizen Kane (1941)
3,"Handmaid's Tale, The (1990)"
4,Desperate Living (1977)
5,Eddie Murphy Raw (1987)
6,Fall (1997)
7,Deathtrap (1982)
8,"Man Without a Past, The (Mies vailla menneisyy..."
9,Glitter (2001)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [31]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [35]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, rerank_user, movie, 'click')
    time.sleep(5)
    

sending event click for Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
sending event click for Zatoichi and the Chest of Gold (ZatÃ´ichi senryÃ´-kubi) (ZatÃ´ichi 6) (1964)
sending event click for 3 Idiots (2009)


Now we can look at how the click events changed the recommendations.

In [36]:
user_recommendations_df

,554,"Horseman on the Roof, The (Hussard sur le toit, Le) (1995)",Zatoichi and the Chest of Gold (ZatÃ´ichi senryÃ´-kubi) (ZatÃ´ichi 6) (1964),3 Idiots (2009)
0,Swept Away (Travolti da un insolito destino ne...,"Nutty Professor, The (1996)","Crying Game, The (1992)","Nutty Professor, The (1996)"
1,Chariots of Fire (1981),Groundhog Day (1993),"Postman, The (Postino, Il) (1994)",Groundhog Day (1993)
2,Citizen Kane (1941),Brazil (1985),"Piano, The (1993)",Brazil (1985)
3,"Handmaid's Tale, The (1990)",Beetlejuice (1988),Like Water for Chocolate (Como agua para choco...,Beetlejuice (1988)
4,Desperate Living (1977),"Nightmare Before Christmas, The (1993)",Jerry Maguire (1996),"Nightmare Before Christmas, The (1993)"
5,Eddie Murphy Raw (1987),"Fugitive, The (1993)","Remains of the Day, The (1993)","Fugitive, The (1993)"
6,Fall (1997),Labyrinth (1986),Groundhog Day (1993),Labyrinth (1986)
7,Deathtrap (1982),True Romance (1993),Sense and Sensibility (1995),True Romance (1993)
8,"Man Without a Past, The (Mies vailla menneisyy...",Star Trek: First Contact (1996),Powder (1995),Star Trek: First Contact (1996)
9,Glitter (2001),"Mask, The (1994)",Leaving Las Vegas (1995),"Mask, The (1994)"


In the cell above, the first column after the index is the user's default recommendations from the "Top pics for you" recommender, and each column after that has as a header of the movie that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much or a lot; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies to see the impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) 

We will create a new helper function to use the personalized ranking campaign, sice the Recommenders already filter out watched content.

In [37]:
def get_new_ranked_recommendations_df_by_static_filter(recommendations_df, user_id, rerank_item_list, filter_arn):
    
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = str(user_id),
        inputList = rerank_item_list,
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['personalizedRanking']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)

    filter_name = filter_arn.split('/')[1]
    new_rec_df = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [38]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_ranked_recommendations_df_by_static_filter(recommendations_df_events, rerank_user, rerank_item_list, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,NaN,Citizen Kane (1941)
1,NaN,Chariots of Fire (1981)
2,NaN,"Rose Tattoo, The (1955)"
3,NaN,Swept Away (Travolti da un insolito destino ne...
4,NaN,"Aviator, The (2004)"
5,NaN,Fall (1997)
6,NaN,After the Wedding (Efter brylluppet) (2006)
7,NaN,"Man Without a Past, The (Mies vailla menneisyy..."
8,NaN,Glitter (2001)
9,NaN,Eddie Murphy Raw (1987)


Now lets send a watch event in for the top 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [39]:
ranked_unwatched_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn = rerank_campaign_arn,
    userId = str(rerank_user),
    inputList = rerank_item_list,
    filterArn = filter_arn)

item_list = ranked_unwatched_recommendations_response['personalizedRanking'][:4]

for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(userId=rerank_user, itemId=item['itemId'], eventType='Watch')
    time.sleep(10)

sending event watch for Citizen Kane (1941)
sending event watch for Chariots of Fire (1981)
sending event watch for Rose Tattoo, The (1955)
sending event watch for Swept Away (Travolti da un insolito destino nell'azzurro mare d'Agosto) (1975)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [40]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_ranked_recommendations_df_by_static_filter(recommendations_df_events, rerank_user, rerank_item_list, filter_arn)
recommendations_df_events

,watched,unwatched
0,Swept Away (Travolti da un insolito destino ne...,Eddie Murphy Raw (1987)
1,Chariots of Fire (1981),"Aviator, The (2004)"
2,Citizen Kane (1941),"Man Without a Past, The (Mies vailla menneisyy..."
3,"Rose Tattoo, The (1955)","Bank Dick, The (1940)"
4,NaN,Desperate Living (1977)
5,NaN,"King's Speech, The (2010)"
6,NaN,Deathtrap (1982)
7,NaN,Adam and Eve (National Lampoon's Adam & Eve) (...
8,NaN,"Handmaid's Tale, The (1990)"
9,NaN,Glitter (2001)


## Batch Recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the Personalized Ranking solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/recommendations-batch.html). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": <S3 input path>}},
    jobOutput = 
       {"s3DataDestination": {"path": <S3 output path>}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for Personalized Ranking, it looks like:


```JSON
{"userId": "891", "itemList": ["27", "886", "101"]}
{"userId": "445", "itemList": ["527", "55", "901"]}
{"userId": "71", "itemList": ["27", "351", "101"]}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"891","itemList":["27","886","101"]},"output":{"recommendedItems":["27","101","886"],"scores":[0.48421,0.28133,0.23446]}}
{"input":{"userId":"445","itemList":["527","55","901"]},"output":{"recommendedItems":["901","527","55"],"scores":[0.46972,0.31011,0.22017]}}
{"input":{"userId":"71","itemList":["29","351","199"]},"output":{"recommendedItems":["351","29","199"],"scores":[0.68937,0.24829,0.06232]}}

```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [41]:
# We will use the same users from before
print (users)
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '", "itemList":'+json.dumps(rerank_item_list)+'}\n')

[242, 557, 554]


In [42]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "242", "itemList":["103245", "7920", "3929", "81845", "90439", "30812", "5156", "72356", "34330", "1957", "5938", "923", "6286", "86059", "2239", "53468", "182823", "4102", "50514", "105468", "8921", "134808", "4775", "1574", "6678"]}
{"userId": "557", "itemList":["103245", "7920", "3929", "81845", "90439", "30812", "5156", "72356", "34330", "1957", "5938", "923", "6286", "86059", "2239", "53468", "182823", "4102", "50514", "105468", "8921", "134808", "4775", "1574", "6678"]}
{"userId": "554", "itemList":["103245", "7920", "3929", "81845", "90439", "30812", "5156", "72356", "34330", "1957", "5938", "923", "6286", "86059", "2239", "53468", "182823", "4102", "50514", "105468", "8921", "134808", "4775", "1574", "6678"]}


Upload the file to S3 and save the path as a variable for later.

In [43]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://131578276461-us-east-1-personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [44]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://131578276461-us-east-1-personalizepocvod/


Now just make the call to kick off the batch export process.

In [45]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = rerank_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-PersonalizedRanking_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [46]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  09:37:29 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: ACTIVE
Import Completed on:  09:48:31 PM


In [47]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,User: 242,User: 557,User: 554
0,Swept Away (Travolti da un insolito destino ne...,"King's Speech, The (2010)",Swept Away (Travolti da un insolito destino ne...
1,Chariots of Fire (1981),Partly Cloudy (2009),Chariots of Fire (1981)
2,Desperate Living (1977),Bright (2017),Citizen Kane (1941)
3,Glitter (2001),Fido (2006),"Handmaid's Tale, The (1990)"
4,Deathtrap (1982),Hop (2011),Desperate Living (1977)
5,"Man Without a Past, The (Mies vailla menneisyy...",Adam and Eve (National Lampoon's Adam & Eve) (...,Eddie Murphy Raw (1987)
6,"Rose Tattoo, The (1955)",Cloudy with a Chance of Meatballs 2 (2013),Fall (1997)
7,Fall (1997),No Way Jose (2015),Deathtrap (1982)
8,"Handmaid's Tale, The (1990)",Fall (1997),"Man Without a Past, The (Mies vailla menneisyy..."
9,"Big Doll House, The (1971)",Glitter (2001),Glitter (2001)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, make a pull request and provide any additional helpful components that may be missing from this collection.

You can choose to head to `04_Operations_Layer.ipynb` to go deeper into ML Ops and what a production solution can look like with an automation pipeline.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `05_Clean_Up.ipynb`.

In [48]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)


In [49]:
print(event_tracker_arn)

arn:aws:personalize:us-east-1:131578276461:event-tracker/d8bc4001


In [50]:
print(batchInferenceJobArn)

arn:aws:personalize:us-east-1:131578276461:batch-inference-job/VOD-POC-Batch-Inference-Job-PersonalizedRanking_1668029848911
